In [222]:
# -*- coding: utf-8 -*-
'''
frx 
2018.6.27
'''

import os
import re

## 读取shell脚本中自定义表名
def shell_defined(path,var):
    dic_s = {}
    with open(path,encoding='latin-1') as f2:
        for k in f2.readlines():
            k = k.upper()
            if var+'=' in k:
                line = k.strip().split('=')
                if "HBK" in line[1]:
                    dic_s[line[0]]=line[1].lower().replace('${hbk}','hbkdb')
                else:
                    dic_s[line[0]]=line[1].lower()
    return dic_s

def all_path(dirname):
    file = open('table_new_new.txt','w')
    num = 0;man=[]
    postfix = set(['sh'])  # 设置要保存的文件格式
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            if apath.split('.')[-1] in postfix: 
                num += 1
                #报编码错误，改成‘latin-1’后通过
                with open(apath,encoding='latin-1') as f:
                    ss = f.read()
                    ss = re.sub(r"#.*\n", " ", ss)  # 去除注释
                    ss = ss.lower().replace('\n'or'\t',' ')  #预处理  根据所有的sh文件内容
                    ss = re.sub(r"\s{2,}", " ", ss)
                    #a = re.findall(r'SQL="(.*?)"log', ss, re.IGNORECASE)
                    #匹配sql语句 一个sql语句为一个处理单元 此处写了半天(ㄒoㄒ)
                    b = re.findall(r'sql[_a-z0-9]*="(.*?)log "sql', ss, re.IGNORECASE)  
                    for i in b:
                        fa=[]
                        word = i.strip().split(' ')
                        #print(word[word.index('insert')+3])
                        # 获得insert后面的表名  insert后面跟的表名都是在insert往后数第三个字符
                        try:
                            if '$' in word[word.index('insert')+3]:
                                t1 = word[word.index('insert')+3].strip('$').strip('{').strip('}').upper()
                                if t1 in dic:
                                    table = dic[t1]
                                else:
                                    dic_shell = shell_defined(apath,t1)
                                    try:
                                        table = dic_shell[t1]
                                    except KeyError:
                                        man.append(apath)
                                        print('该sh文件中存在自定义表名，需手工处理',apath,t1,dic_shell)
                            else:
                                table = word[word.index('insert')+3]

                        #获得from的所有索引，如果下一个是表名则获取，$的需要查表，查不到说明是shell脚本中自定义的，抛异常，需手工处理
                            for j,item in enumerate(word): 
                                if item=='from' or item=='join':
                                    if '$' in word[j+1]:
                                        t2 = word[j+1].strip('$').strip('{').strip('}').upper()
                                        #print('@@',word[j+1])
                                        if t2 in dic:
                                            fa.append(dic[t2])
                                        else:
                                            dic_shell = shell_defined(apath,t2)
                                            try:
                                                fa.append(dic_shell[t2])
                                            except KeyError:
                                                man.append(apath)   
                                                print('该sh文件中存在自定义表名，需手工处理',apath,t2,dic_shell)
                                    elif  '_' in word[j+1]:
                                         fa.append(word[j+1])
                            if len(fa)>0:
                                file.write(apath + '\t' + table + '\t'+ str(fa) + '\n')
                            table = 'hahaha';fa=[]
                        except ValueError:
                            num += 0  
    with open('select_new_new.txt','w') as f1:
        for i in list(set(man)):
            f1.write(i+'\n')                   
    file.close()
    return num,len(man)

if __name__ == '__main__':
    dirpath = "./shell"  # 指定根目录
    nn,m = all_path(dirpath)
    print('sh数量',nn)
    print('需要二次处理的文件数量',m)
    

该sh文件中存在自定义表名，需手工处理 ./shell\application\jrms\send.sh "${TBL_JRMS_DETAIL}" {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\jrms\S_LD_D_JRMS_MCHNT_DETAIL.sh "${TBL_JRMS_DETAIL}" {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\jrms\test.sh "${TBL_JRMS_DETAIL}" {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_DATA_MPOS_INS.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_ACHNL_ACQINS.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_ACHNL_MCHNT.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_CLOUDPAY_INS.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_CLOUDPAY_MCHNT.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_CUPS_ACQINS.sh ($SQL1 {}
该sh文件中存在自定义表名，需手工处理 ./shell\application\oda\outlierAnalysis\S_LD_D_ODAS_DRILL_MODEL_CUPS_INTNL

In [ ]:
with open(apath,'r') as f2:
    for i in f.readlines():
        if len(re.find('=.*..*_',i))>0:
            

In [172]:
dic={}
with open('table.conf',encoding='utf-8') as f:
    for i in f.readlines():
        
        i = i.replace('export ','')
        l = i.strip().split('=')
        if len(l)>1:
            dic[l[0]]=l[1].strip(' ')
dic

{'DTDTRS_BSL_ALL_CHNL_FIRST_TRANS_CARD': 'hbkdb.dtdtrs_bsl_all_chnl_first_trans_card',
 'DTDTRS_BSL_BILL_POS_TRANS_LOG': 'hbkdb.dtdtrs_bsl_bill_pos_trans_log',
 'DTDTRS_BSL_UV_ACTIVE_CARD_TMP': 'hbkdb.dtdtrs_bsl_uv_active_card_tmp',
 'DTDTRS_BSL_UV_ONLINE_TMP': 'hbkdb.dtdtrs_bsl_uv_online_tmp',
 'DTDTRS_DTL_ACHIS_MA_CUPS': 'hbkdb.dtdtrs_dtl_achis_ma_cups',
 'DTDTRS_DTL_AM': 'hbkdb.dtdtrs_dtl_am',
 'DTDTRS_DTL_CUPS': 'hbkdb.dtdtrs_dtl_cups',
 'DTDTRS_DTL_CUPS_EXT': 'hbkdb.dtdtrs_dtl_cups_ext',
 'DTDTRS_DTL_CUPS_FEE': 'hbkdb.dtdtrs_dtl_cups_fee',
 'DTDTRS_DTL_CUPS_GZIP': 'hbkdb.dtdtrs_dtl_cups_gzip',
 'DTDTRS_DTL_CUPS_JOINED': 'hbkdb.dtdtrs_dtl_cups_joined',
 'DTDTRS_DTL_CUPS_MR': 'hbkdb.dtdtrs_dtl_cups_mr',
 'DTDTRS_DTL_MCHNT': 'hbkdb.dtdtrs_dtl_mchnt',
 'DTDTRS_DTL_SAS': 'hbkdb.dtdtrs_dtl_sas',
 'DVDTRS_DTL_CUPS': 'hbkdb.dvdtrs_dtl_cups',
 'HIVE_DETAIL_CUPS_FEE': 'hbkdb.rtdtrs_cups_fee',
 'HIVE_DETAIL_CUPS_SSA': 'hbkdb.tbl_hbk_dtl_ssa_trans_log',
 'HIVE_DETAIL_CUPS_SSA_FEE': 'hbkdb.rtd

In [219]:
def shell_defined(path,var):
    dic_s = {}
    with open(path,encoding='latin-1') as f2:
        for k in f2.readlines():
            k = k.upper()
            if var+'=' in k:
                line = k.strip().split('=')
                if "HBK" in line[1]:
                    dic_s[line[0]]=line[1].lower().replace('${hbk}','hbkdb')
                else:
                    dic_s[line[0]]=line[1].lower()
    return dic_s
apath = '3.sh';man=[]
file = open('te.txt','w')
with open(apath,encoding='latin-1') as f:
    ss = f.read()
    ss = re.sub(r"#.*\n", " ", ss)  # 去除注释
    ss = ss.lower().replace('\n'or'\t',' ')  #预处理  根据所有的sh文件内容
    ss = re.sub(r"\s{2,}", " ", ss)
    #a = re.findall(r'SQL="(.*?)"log', ss, re.IGNORECASE)
    #匹配sql语句 一个sql语句为一个处理单元 此处写了半天(ㄒoㄒ)
    b = re.findall(r'sql[_a-z0-9]*="(.*?)log "sql', ss, re.IGNORECASE)  
    print(len(b))
    for v in range(len(b)):
        print('***')
        print(b[v])
    for i in b:
        fa=[]
        word = i.strip().split(' ')
        #print(word[word.index('insert')+3])
        # 获得insert后面的表名  insert后面跟的表名都是在insert往后数第三个字符
        try:
            if '$' in word[word.index('insert')+3]:
                t1 = word[word.index('insert')+3].strip('$').strip('{').strip('}').upper()
                print('t1',t1)
                if t1 in dic:
                    table = dic[t1]
                else:
                    dic_shell = shell_defined(apath,t1)
                    try:
                        table = dic_shell[t1]
                    except KeyError:
                        man.append(apath)
                        print('该sh文件中存在自定义表名，需手工处理',apath,t1,dic_shell)
            else:
                print('t1',t1)
                table = word[word.index('insert')+3]

        #获得from的所有索引，如果下一个是表名则获取，$的需要查表，查不到说明是shell脚本中自定义的，抛异常，需手工处理
            for j,item in enumerate(word): 
                if item=='from' or item=='join':
                    if '$' in word[j+1]:
                        t2 = word[j+1].strip('$').strip('{').strip('}').upper()
                        print('t2',t2)
                        #print('@@',word[j+1])
                        if t2 in dic:
                            fa.append(dic[t2])
                        else:
                            dic_shell = shell_defined(apath,t2)
                            try:
                                fa.append(dic_shell[t2])
                            except KeyError:
                                man.append(apath)   
                                print('该sh文件中存在自定义表名，需手工处理',apath,t2,dic_shell)
                    elif  '_' in word[j+1]:
                         fa.append(word[j+1])
            print('fa',fa)
            if len(fa)>0:
                file.write(apath + '\t' + table + '\t'+ str(fa) + '\n')
        except ValueError:
            num += 0  

4
***
 $(setphonelookup) $(setiplookup) $(defhivejobname "kylin_cupsåºäºcups_transform") insert overwrite table hbkdb.kaohe_yidong_xianshang_mid partition(hp_settle_mon='$datemon') select intnl ,prod ,org_mcc ,mchnt_cd ,trans_chnl ,app_ins_id_cd ,sum(shoujipay_jinchang_num) ,sum(shoujipay_yuancheng_num) ,sum(yidong_wuka_num) ,sum(erweima_num) ,sum(shoujipay_xianshang_at) ,sum(xianshangwuka_at) from ( select case when prod in ('nfcçº¿ä¸-applepay','nfcçº¿ä¸-hce','nfcçº¿ä¸-ä¸æpaymst','nfcçº¿ä¸-ä¸æpayic','nfcçº¿ä¸-åä¸º','nfcçº¿ä¸-å°ç±³','nfcçº¿ä¸-å¶ä»') then acq_intnl_nm when prod in ('nfcçº¿ä¸-applepay','nfcçº¿ä¸-hce','nfcçº¿ä¸-ä¸æpaymst','nfcçº¿ä¸-ä¸æpayic','nfcçº¿ä¸-åä¸º','nfcçº¿ä¸-å°ç±³','nfcçº¿ä¸-å¶ä»') then nvl(ach_reg_intnl_nm,acq_intnl_nm) when prod in ('æ å¡-è®¤è¯æ¯ä»','æ å¡-å¿«æ·æ¯ä»','æ å¡-ç½é¶æ¯ä»','æ å¡-ä¿¡ç¨å¡æ¯ä»','æ å¡-è®¤è¯æ¯ä»2.0','æ å¡-åå°æ¯ä»','å¨æ¸ éæ å¡å¶ä»') then nvl(ach_reg_intnl_nm,acq_i

In [165]:
file = open('result.txt','w');apath='a.sh';dic={'a':1}
with open('1.sh',encoding='utf-8') as f1:
    ss = f1.read().lower().replace('\n'or'\t',' ')  #预处理  根据所有的sh文件内容
    ss = re.sub(r"\s{2,}", " ", ss)
    #a = re.findall(r'SQL="(.*?)"log', ss, re.IGNORECASE)
    #匹配sql语句 一个sql语句为一个处理单元 此处写了半天(ㄒoㄒ)
    b = re.findall(r'sql[0-9]*="(.*?)log "sql', ss, re.IGNORECASE)  
    for i in b:
        fa=[]
        word = i.strip().split(' ')
        #print(word[word.index('insert')+3])
        # 获得insert后面的表名  insert后面跟的表名都是在insert往后数第三个字符
        if '$' in word[word.index('insert')+3]:
            try:
                table = dic[word[word.index('insert')+3].strip('@')]
            except KeyError:
                print('该sh文件中存在自定义表名，需手工处理',apath)
                continue
        else:
            table = word[word.index('insert')+3]
        #获得from的所有索引，如果下一个是表名则获取，$的需要查表，查不到说明是shell脚本中自定义的，抛异常，需手工处理
        for j,item in enumerate(word): 
            if item=='from':
                if '$' in word[j+1]:
                    #print('@@',word[j+1])
                    try:
                        fa.append(dic[word[j+1].strip('@')])
                    except:
                        print('该sh文件中存在自定义表名，需手工处理',apath)
                        break
                elif  '_' in word[j+1]:
                     fa.append(word[j+1])
        #table.append(word[word.index('insert')+3]) 
        file.write(apath + '\t' + table + '\t'+ str(fa) + '\n')
file.close()

该sh文件中存在自定义表名，需手工处理 a.sh
@@ $dvdtrs_dtl_cups
该sh文件中存在自定义表名，需手工处理 a.sh


In [68]:
sheet = []
with open('new_t.txt',encoding='utf-8') as f:
    try:
        s = f.read()
        if 'from'in s or 'FROM'in s:
            for i in f.readlines():
                line1 = i.strip().split('\t')
                print(line1)
                if 'from' in line1 or 'FROM' in line1:
                    if len(line1)<2:
                        print('***')
                if 'insert' in line1 or 'INSERT' in line1:
                    if len(line1)==1:
                        print('@@@')
    except UnicodeDecodeError:
        num += 0

In [253]:
filte = ['bushuju','tmp','temp','backup','test','wormhole']
f = open('table_new_new.txt','r')
filename = [];table=[];fa=[]
for i in f.readlines():
    line = i.strip().split('\t')
    flag = 0
    for j in filte:
        if j in line[0]:
            flag += 1
    if flag>0:
        continue
    if line[1].lower()=='directory':
        continue
    filename.append(line[0])
    if '$' in line[1]:
        line[1] = dic[line[1].strip('"').strip('$').strip('{').strip('}').upper()]
    table.append(line[1])
    line[2] = eval(line[2])
    if '$' in str(line[2]):
        for j,item in enumerate(line[2]):
            if '$' in item:
                line[2][j] = dic[line[2][j].strip('$').strip('{').strip('}').upper()]
    fa.append(line[2])
                
    

In [256]:
import pandas as pd
dd = pd.DataFrame()
dd['filename']=filename
dd['table']=table
dd['fa']=fa
dd.to_csv('RE.csv',index=0,encoding='utf8')

In [259]:
d1 = dd.drop_duplicates()

TypeError: unhashable type: 'list'

In [277]:
t = dd.drop(['filename'],axis=1)
edges = [(j.strip("'"),t.iloc[i,0].strip("'")) for i in range(len(t)) for j in eval(str(t.iloc[i,1])) ]
edges

[('hbkdb.dvdtrs_dtl_cups',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_trans',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_ins',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_mcc_old',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_ins',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.dvdtrs_dtl_cups',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_ins',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_trans',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_ins',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.rtapam_dim_mcc_old',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans',
  'hbkdb.stmtrs_bsl_account_mchnt_trans_disc_rank_trans'),
 ('hbkdb.stmtrs_bsl

In [278]:
new_edges = []
for i in edges:
    if i[0]!=i[1]:
        new_edges.append(i)
len(new_edges)

5365

In [279]:
new_edges = list(set(new_edges))
len(new_edges)

1515

In [280]:
print(new_edges)

[('hbkdb.tbl_hbk_dim_ins', 'hbkdb.stmtrs_mnt_logout'), ('gl_hisdb.tbl_mchis_mgm_term_inf_all_seq', 'hbkdb.stdtrs_bsl_term_unique_total_all'), ('hbkdb.stmtrs_dal_active_card_analyse_export', 'hbkdb.stmtrs_dal_active_card_analyse_info_export'), ('hbkdb.stmtrs_scl_mchnt_stat', 'hbkdb.stmtrs_app_mchnt_detect_register_tmp'), ('gl_hisdb.tbl_mchis_mgm_indirect_mchnt_inf', 'hbkdb.stdtrs_scl_mchnt_clean_base'), ('hbkdb.rtapam_dim_ins', 'hbkdb.stdtrs_bsl_detect_yyzx_achis_verify'), ('default.tbl_common_his_trans_inquiry', 'hbkdb.rtdtrs_dtl_cups'), ('hbkdb.rtapam_dim_hr_emp_inf', 'hbkdb.stdtrs_bsl_hr_emp_prod_trans'), ('hbkdb.stdtrs_bsl_scb_daifu_trans', 'hbkdb.stdtrs_bsl_scb_daifu_trans_total'), ('hbkdb.ml_sample_yishi_big1', 'hbkdb.ml_sample_yishi_big'), ('hbkdb.rtapam_dim_mcc', 'hbkdb.ststrs_bsl_active_card_acq_branch_season2'), ('hbkdb.dvdtrs_dtl_cups', 'hbkdb.stdtrs_bsl_udc_qrpay_mid'), ('gl_hisdb.tbl_chhis_ubp_order', 'hbkdb.stdtrs_bsl_hr_new_wallet_trans_dtl1'), ('hbkdb.stmtrs_bsl_bcci_car

In [281]:
table_set=[]
for i in edges:
    table_set.append(i[0].strip("'"))
    table_set.append(i[1].strip("'"))
table_set = set(table_set)
len(table_set)

820

In [282]:
table_set

{'acqins_range',
 'cloud_acq_tmp',
 'cloud_iss_tmp',
 'cloud_tmp',
 'default.tbl_common_his_trans',
 'default.tbl_common_his_trans_inquiry',
 'gl_hisdb.tbl_achis_crd_mchnt_bind_relation',
 'gl_hisdb.tbl_achis_mgm_qrc_acq_inf)f',
 'gl_hisdb.tbl_achis_mgm_qrc_app_inf',
 'gl_hisdb.tbl_achis_sor_trans_flow',
 'gl_hisdb.tbl_achis_trans_flow',
 'gl_hisdb.tbl_achis_trans_flow_xb',
 'gl_hisdb.tbl_achis_upop_data_dic',
 'gl_hisdb.tbl_chhis_acc_cdhd_card_bind_inf',
 'gl_hisdb.tbl_chhis_acc_cdhd_point_addup_his_dtl',
 'gl_hisdb.tbl_chhis_acc_cdhd_pri_acct_inf',
 'gl_hisdb.tbl_chhis_acc_code_pay_tran_dtl',
 'gl_hisdb.tbl_chhis_acc_online_point_trans_his_inf',
 'gl_hisdb.tbl_chhis_mgm_acc_trans_his_dtl',
 'gl_hisdb.tbl_chhis_mgm_bill_sub_order_detail_inf',
 'gl_hisdb.tbl_chhis_mgm_swt_log',
 'gl_hisdb.tbl_chhis_mgm_tax_rebate_record',
 'gl_hisdb.tbl_chhis_mgm_trans_day',
 'gl_hisdb.tbl_chhis_mksvc_order',
 'gl_hisdb.tbl_chhis_mnsvc_business_instal_info',
 'gl_hisdb.tbl_chhis_ubp_order',
 'gl_hisdb.

In [289]:
pd.DataFrame(list(table_set)).to_csv('node.csv',index=0,encoding='utf8')
pd.DataFrame(new_edges).to_csv('edges.csv',index=0,encoding='utf8')